# 단어TDM(Term-Document Matrix)
---
형태소 분석을 바탕으로 TDM(단어 문서 행렬)만들기

Vectorizer하여 단어 빈도 분석하기

(Count: 단순 빈도 분석 / TF-IDF: 단어 희소가치 분석)

# 뉴스 기사 TDM

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("한국방송통신전파진흥원_전파누리_최신뉴스_20210129.csv",
                encoding='cp949', engine='python')

In [3]:
df.shape

(540, 6)

In [4]:
df.columns

Index(['제목', '내용', '조회수', '첨부파일명', '첨부파일 링크', '출처'], dtype='object')

In [5]:
df['내용'].head()

0    한국전자파학회가 발간하는 영문 학술지 'JEES'가 국제 과학기술 논문 인용색인 확...
1    미국 AT&T가 39㎓ 밀리미터파 대역을 활용한 '5G플러스(5G+)' 서비스를 상...
2    우리나라는 경쟁 활성화와 통신비 인하를 목표로 2010년부터 7차례 제4 이동통신사...
3    과학기술정보통신부가 SK텔레콤 2세대(2G) 이동통신 가입자들의 민원 분석과 기지국...
4    과학기술정보통신부가 공공용 주파수를 효율적으로 수급·관리하기 위한 업무처리지침을 마...
Name: 내용, dtype: object

#### count vectorizer

In [6]:
from konlpy.tag import Komoran

In [7]:
tagger = Komoran()

In [8]:
def get_nouns(text):
    nouns = tagger.nouns(text)
    nouns = [word for word in nouns if len(word) > 1]
    return nouns

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
cv = CountVectorizer(max_features=1000, tokenizer = get_nouns)

In [11]:
tdm = cv.fit_transform(df['내용'])

In [12]:
cv.get_feature_names()[-10:]

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['회의', '회장', '획득', '효과', '효율', '후보', '후속', '훈련', '휴대', '휴대폰']

In [13]:
tdm

<540x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 16137 stored elements in Compressed Sparse Row format>

#### Tfidf vectorizer

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
tfidf = TfidfTransformer()

In [16]:
tdm2 = tfidf.fit_transform(tdm)

#### 단어 비교

In [17]:
words = cv.get_feature_names()

In [18]:
for i, n in sorted(zip(tdm[0].indices, tdm[0].data)):
    print(words[i], n)

결과 1
과학기술 1
구성 1
국제 3
기준 2
발간 1
우수 1
전자파 1
점수 1
통과 1
평가 1
품질 1
학회 1
한국 1
항목 2
확장 1


In [19]:
for i, n in sorted(zip(tdm2[0].indices, tdm2[0].data)):
    print(words[i], n)

결과 0.1619983121646564
과학기술 0.09401856636507744
구성 0.16867880285062742
국제 0.43704825177749196
기준 0.3041618237966519
발간 0.2317471773380904
우수 0.22596027308497585
전자파 0.19592107190255614
점수 0.2384276680240614
통과 0.19871800403304712
평가 0.15307721799666532
품질 0.1672577769673222
학회 0.25599947426739555
한국 0.1672577769673222
항목 0.4634943546761808
확장 0.2317471773380904


#### 저장

In [20]:
import joblib

In [21]:
with open('ai_news_tfidf.pkl', 'wb') as f:
    joblib.dump(
        {'vectorizer' : tfidf, 'tdm' :tdm2},
        f
    )

In [22]:
with open('ai_news_count.pkl', 'wb') as f:
    joblib.dump(
        {'vectorizer' : cv, 'tdm' :tdm},
        f
    )

# 아마존 제품 리뷰 TDM

In [23]:
df_2 = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)

In [24]:
df_2.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [25]:
content = df_2[0]

In [26]:
sentiment = df_2[1]

In [27]:
df_2.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
 # stop_words 기존에 만들어져 있는 불필요한 단어set 필터링 영어만 있음
tfidf = TfidfVectorizer(stop_words='english',
                       lowercase=True,
                       max_features=1000)

In [30]:
tdm3 = tfidf.fit_transform(content)

In [31]:
tdm3

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 4060 stored elements in Compressed Sparse Row format>

In [32]:
tfidf.get_feature_names()[-10:]

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['worthless',
 'worthwhile',
 'wouldn',
 'wow',
 'wrong',
 'wrongly',
 'year',
 'years',
 'yell',
 'yes']

In [33]:
import joblib

In [34]:
with open('amazon.pkl', 'wb') as f:
    joblib.dump(
        {'vectorizer' : tfidf, 'tdm': tdm3, 'sentiment' : sentiment},
        f
    )